디자인 원칙과 단위 테스트
- 먼저 단위테스트란?
=> 다른 코드의 일부분이 유효한지를 검사하는 코드

=> 단위테스트는 소프트웨어의 핵심이 되는 필수 기능으로써, 단순히 "핵심"을 검증하는 게 아님.

단위 테스트(Unit test)의 특징


- 격리 : 단위 테스트는 다른 외부 에이전트와 완전히 독립적이어야 하며 비즈니스 로직에만 집중해야 함
- 성능 : 신속하게 실행되어야 하며, 반복적으로 여러 번 실행될 수 있도록 설계해야 한다.
- 자체 검증 : 단위 테스트의 실행만으로 결과를 결정할 수 있어야 한다. 단위 테스트를 위한 추가 단계가 없어야함(일반적으로 실패하면 오류코드를/ 성공하면 점을/ 실패하면  F/ 예외가 있으면 E)

자동화된 테스트의 다른 형태

- 단위 테스트는 함수 또는 메서드와 같은 "작은 단위"를 확인하귀 위한 것
- 즉, 최대한 "자세하게" 코드를 검사하는 것이 목적임
- 따라서, 더 큰 개념인 클래스를 검사하려면 "단위테스트 집합"인 테스트 스위트를 사용한다.
- 단위 테스트의 방법은 다양하고, 모든 오류를 잡을 수 있는 것도 아님
- 통합/인수 테스트 라는 것도 있는데 이것들은 단위테스트와 관련된 중요 특성인 "속도"를 잃게 된다.
- 따라서, 좋은 개발환경은 전체 테스트 스위트에서 수정이 생길때마다 단위 테스트 및 리팩토링을 해야함
- 그럼에도 불구하고 **실용성이 이상보다 우선**이다.


## 1. 단위 테스트와 애자일

- 작은 성공을 지속적으로 반복 → 빠른 피드백을 받아 수정 반복을 하는 것
- 솔리드 원칙을 지키고, 개방/폐쇄 원칙에 따르는 컴포넌트를 만들었고 리팩토링이 쉬워 변화에 대응가능할때

    → 버그가 발생하지 않으려면?

    → 기존 기능이 보존되었는지? 문제가 없는지?

- 이 질문의 근거가 단위 테스트가 될 수 있음.
- 즉, 단위 테스트는 코드가 우리가 기대한 것처럼 동작한다는 확신을 주는 안전망


## 2. 단위 테스트와 소프트웨어 디자인

- 좋은 소프트웨어는 테스트 가능한 소프트웨어

In [2]:
class MetricsClient:
		"""타사 지표 전송 클라이언트"""
		def sent(self, metric_name, metric_value):
				if not isinstance(metric_name, str):
						raise TypeError("metric_name으로 문자열 타입을 사용해야 함")

				if not isinstance(metric_value, str):
						raise TypeError("metric_value로 문자열 타입을 사용해야 함")

				logger.info(f"{metric_name} 전송 값 = {metric_value})

class Process:

		def __init__(self):
				self.client = MetricsClient() # 타사 지표 전송 클라이언트

		def process_iterations(self, n_iterations):
				for i in range(n_interations):
						result = self.run_process()
						self.client.sent("iteration.".format(i), result)

SyntaxError: EOL while scanning string literal (<ipython-input-2-0dc52991e1a0>, line 10)

- 위 코드에서는 타사 지표 전송 클라이언트의 파라미터가 문자열 타입이어야 한다는 요구사항이 있음
- 타사에서 제공하는 라이브러리는 직접 제어가 불가능 하므로, 리팩토링을 여러번 하더라도 단위 테스트를 통해 버그가 재현되지 않음을 확인해야 한다.


- 필요한 부분만 테스트 하기위해 main 메서드에서 client를 직접 다루지 않고 래퍼 메서드에 위임함
- 메인 코드에 직접 단위 테스트를 작성하면 가장 중요한 속성인 추상화를 하지 못하게 됨

In [ ]:
class WrappedClient:
		def __init__(self):
				self.client = MetricsClient()

		def send(self, metric_name, metric_value):
				return self.client.send(str(metric_name), str(metric_value))
                                #여기 sent가 아니라 sent가 맞나요?
class Process:
		def __init__(self):
				self.client = WrappedClient()
                
                #나머지는 유지
                

In [ ]:
- Mock(흉내내다): 의존성 객체를 대신하여 시뮬레이션 하는 역할
- 어떤 종류의 타입에도 사용할 수 있음

In [5]:
import unittest
from unittest.mock import Mock

class TestWrappedClient(unittest.TestCase):
		def test_sent_converts_types(self):
				wrapped_client = WrappedClient()
				wrapped_client.client = Mock()
				wrapped_client.sent("value", 1)
				wrapped_client.client.send.assert_called_with("value", "1")

# MOCK
https://www.daleseo.com/python-unittest-mock/
- mocking은 소외 mock이라고 불리는 가짜 객체를 생성하는 것부터 시작함}

먼저 호출되었을 때 특정 값을 리턴하는 mock 객체는 return_value 옵션을 이용해서 생성할 수 있음

In [6]:
mock = Mock(return_value='Hello, Mock!')
mock()

'Hello, Mock!'

반면에 호출되었을 때 예외가 발생하는 mock 객체는 side_effect 옵션을 이용해서 생성할 수 있음

In [7]:
mock = Mock(side_effect=Exception('Oops!'))
mock()

Exception: Oops!

side_effect 옵션에 리스트를 넘기면 mock 객체가 호출될 때 마다 매 번 다른 값을 리턴할 수도 있음.

In [13]:
mock = Mock(side_effect=[1, 2, 3])
mock()
mock()
mock()


3

In [14]:
mock = Mock(side_effect=[1, 2, 3])
mock()
mock()
mock()
mock()

StopIteration: 

side_effect 옵션에 함수를 넘기면 mock 객체를 호출했을 때 주어진 인자에 따라 다른 값을 리턴할 수 있음,.


In [15]:
mock = Mock(side_effect=lambda x: x * 10)
mock(3)

30

return_value와 side_effect 옵션은 꼭 Mock() 생성자의 인자로 넘어갈 필요는 없음. 
다음과 같이 mock 생성 이후에도 얼마든지 이 옵션 값은 바꿀 수가 있다.



In [17]:
mock = Mock()
mock.return_value=34
mock()

34

 위와 같이 함수처럼 바로 호출을 할 수도 있지만, 객체처럼 속성도 가질 수 있는데 각 속성은 새로운 mock이 됨. 따라서 다음과 같이 특정 속성에 값을 할당해 수도 있고, 특정 메서드의 리턴 값을 지정해줄 수도 있음

In [19]:
mock = Mock()
mock.attribute = 'ATTRIBUTE'
mock.attribute


'ATTRIBUTE'

In [20]:

mock.method.return_value = 'METHOD RETURN VALUE'
mock.method()


'METHOD RETURN VALUE'

이렇게 mock 객체의 속성이나 메서드도 또 다른 mock 객체가 된다는 사실을 이용하여 더 유연하게 사용할 수 있다고 하네요 

# mock로 객체 검증하기

- assert_called() 메서드는 해당 mock이 호출된 이력이 있는지를 검증할 때 쓰임
- mock을 한 번도 호출하지 않고 assert_called() 메서드를 호출하면 예외가 발생함.

In [21]:
from unittest.mock import Mock
mock = Mock()
mock.assert_called()


AssertionError: Expected 'None' to have been called.

In [22]:
from unittest.mock import Mock
mock = Mock()
mock()
mock.assert_called()


assert_called_once() 메서드는 해당 mock이 단 한 번 호출되었는지 검증할 때 쓰임

In [23]:
mock.assert_called_once()

In [24]:
mock()
mock.assert_called_once()

AssertionError: Expected 'mock' to have been called once. Called 2 times.

assert_called_with() 메서드를 사용하면 해당 mock이 호출되었을 때 어떤 인자가 넘어왔는지까지도 검증할 수 있음.

In [25]:
mock = Mock()
mock('A', B='C')
mock.assert_called_with('A', B='C')

assert_not_called() 메서드는 지금까지와 반대로 해당 mock이 호출된 적이 없는지 검증할 때 쓰임.

In [28]:
mock = Mock()
mock.assert_not_called()
mock()


<Mock name='mock()' id='2513548014728'>

In [29]:
mock.assert_not_called()

AssertionError: Expected 'mock' to not have been called. Called 1 times.

# 테스트의 경계 정하기

- 무엇을 테스트할 지 주의하지 않으면 계속 테스트만 하고 시간 버릴 수 있다
- 테스트의 범위는 우리가 작성한 코드의 범위까지
- 외부 라이브러리 모듈과 같은 의존성까지 확인해야한다면 진짜 끝도 없을 것임
- 결론적으로, 외부라이브러리 쓸때는 올바른 파라미터 넣었을 때 정상적으로 호출된다는 것까지만 확인해주면 된다.
- 

여기까지?


# 테스트를 위한 프레임 워크와 도구
- 거의 모든 시나리오를 다룰 수 있는 두가지 도구 소개

## 단위테스트 프레임워크와 라이브러리
- 테스트 시나리오는 unittest라이브러리만으로 충분
- 외부 시스템에 연결하는 등의 "의존성"이 많은 경우테스트 케이스를 파라미터화 할 수 있는 픽스쳐라는 패치 객체가필요=> pytest

Merge Request에 대해 코드 리뷰를 도와주는 간단한 버전 제어 도구

- 전제조건
    - 한 명 이상의 사용자가 변경 내용에 동의 하지 않은 경우 머지 리퀘스트가 거절(reject)된다.
    - 아무도 반대하지 않은 상태에서 두 명 이상의 개발자가 동의하면 해당 머지 리퀘스트는 승인 된다
    - 이외의 상태는 보류 상태이다

코드는 다음과 같다.

In [30]:
from enum import Enum

class MergeRequestStatus(Enum):
    APPROVED = "approved"
    REJECTED = "rejected"
    PENDING = "pending"

class MergeRequest:
    def __init__(self):
        self._context = {
            "upvotes": set(), 
            "downvotes": set(),
				}

@property
def status(self):
    if self._context["downvotes"]:
        return MergeRequestStatus.REJECTED
    elif len(self._context["upvotes"]) >= 2:
        return MergeRequestStatus.APPROVED
    return MergeRequestStatus.PENDING

def upvote(self, by_user):
    self._context["downvotes"].discard(by_user)
    self._context["upvotes"].add(by_user)

def downvote(self, by_user):
    self._context["upvotes"].discard(by_user)
    self._context["downvotes"].add(by_user)



### [unittest]

unittest모듈은 모든 종류의 테스트를 작성할 수 있는 풍부한 API를 제공하므로 단위 테스트를 시작하기에 훌륭한 선택이다. 

표준라이브러리라 사용이 편리하다.

unittest 모듈은 자바의 JUnit을 기반으로 한다.(Smalltalk 기반으로 만들어졌으며, 객체지향적)

테스트는 객체를 사용해 작성되며 클래스의 시나리오 별로 테스트를 그룹화하는 것이 일반적이다.

단위테스트 정의 방법:

1. unittest.TestCase를 상속
2. 테스트 클래스 생성
3. 메서드에 테스트할 조건을 정의

메서드는 test_ 로 시작해야하며, 예시에서는 unittest.TestCase에서 상속받은 메서드를 사용하여 체크하려는 조건이 참인지 확인하면 된다.

In [ ]:

class TestMergeRequestStatus(unittest.TestCase):
    
    def test_simple_rejected(self):
        merge_request = MergeRequest()
        merge_request.downvote("maintainer")
        self.assertEqual(merge_request.status, MergeRequestStatus.REJECTED)

    def test_just_created_is_pending(self):
        self.assertEqual(MergeRequest().status, MergeRequestStatus.PENDING)
    
    def test_pending_awaiting_review(self):
        merge_request = MergeRequest()
        merge_request.upvote("core-dev")
        self.assertEqual(merge_request.status, MergeRequestStatus.PENDING)

    def test_approved(self):
        merge_request = MergeRequest()
        merge_request.upvote("dev1")
        merge_request.upvote("dev2")
        self.assertEqual(merge_request.status, MergeRequestStatus.APPROVED)


ssertEquals(actual, expected,[, message] : 예상값과 실제 값을 비교

사용자가 머지  리퀘스트를 종료할 수 있도록 함

병합을 종료하면 더이상 투표 할 수 없다.

누군가가 종료된 머지 리퀘스트에 투표를 시도하면 예외를 발생시키도록 함

In [32]:
class MergeRequest:
    def __init__(self):
        sef._context = {
            "upvotes": set(),
            "downvotes": set(),
        }
        self._status = MergeRequestStatus.OPEN
    
    def close(self):
        self._status = MergeRequestStatus.CLOSED## 새로 추가된 매써드
    ... 
    def _cannot_vote_if_closed(self): 
        if self._status == MergeRequestStatus.CLOSED:
            raise MergeRequestException("종료된 머지 리퀘스트에 투표할 수 없음")
    
    def upvote(self, by_user):
        self._cannot_vote_if_closed()

        self._context["downvotes"].discard(by_user)
        self._context["upvotes"].add(by_user)

    def downvote(self, by_user):
        self._cannot_vote_if_closed()

        self._context["upvotes"].discard(by_user)
        self._context["downvotes"].discard(by_user)

유효성 검사가 실제로 작동하는 지 확인하기 위해 assertRaise, assertRaisesRegex 메서드 사용

In [ ]:

# 제공한 예외가 실제로 발생하는지 확인
# 두번째 파라미터로 호출 가능한 객체를 전달하고 나머지 파라미터에 호출에 필요한 파라미터를(*args와 **kwargs) 전달
# assertRaises(exc, fun, *args, **kwds)
def test_cannot_upvote_on_closed_merge_request(self):
    self.merge_request.close()
    self.assertRaises(
        MergeRequestException, self.merge_request.upvote, "dev1"
    )

# 동일한 방식으로 처리하지만 발생한 예외의 메시지가 제공된 정규식과 일치하는지 확인
# 예외가 발생했지만 정규 표현식과 일치하지 않는 다른 메시지가 있는 경우에도 테스트에 실패
# assertRaisesRegex(exc, r, fun, *args, **kwds)
def test_cannot_downvote_on_closed_merge_request(self):
    self.merge_request.close()
    self.assertRaisesRegex(
        "종료된 머지 리퀘스트에 투표할 수 없음",
        self.merge_request.downvote,
        "dev1",
    )


예외 발생여부 뿐만 아니라 오류 메시지도 잘 설정해서 확인하면, 우연히 같은 타입의 예외가 발생했으나 실제로는 다른 원인에 의한 경우를 제외 할 수 있다.